In [203]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [204]:
df = pd.read_csv("iris.csv")
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [205]:
df[["variety"]].value_counts()

variety   
Setosa        50
Versicolor    50
Virginica     50
Name: count, dtype: int64

In [206]:
## Function for loading the data and train test split

'''
--> Enter the name of the file as a string for the variable -- "csv_file" 
--> Enter the name of the "target variable" as a string for the variable -- "target_variable"
--> train_size , test_size -- The ratio you want to split the data (80 ,20)

'''

def data_preprocessing(csv_file,train_size,test_size,The_target_variable):
    
    # load the data using pandas
    
    df = pd.read_csv(csv_file)
    
    df["variety"] = df["variety"].replace({"Setosa": 0, "Versicolor": 1, "Virginica": 2})
    
    print(f"The shape of your dataset is : {df.shape}")
    
    total_no_of_rows = df.shape[0]
    total_no_of_columns = df.shape[1]
    
    # The input features(dependent features) & The target variable(independent feature)
    
    X = df.drop(The_target_variable,axis = 1)
    y = df[[The_target_variable]]
    
    # Train_test_split
    
    X = df.drop(The_target_variable, axis=1)
    y = df[The_target_variable]
    
    # Train-test split
    train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=train_size/100, test_size=test_size/100, random_state=42)
    
    print(f"The shape of your traing data set is: {train_X.shape}")
    print(f"The shape of your testing data set is : {test_X.shape}")
    
    ## Convert the pandas dataframe to to Numpy array to do model building
    
    train_X_arr = train_X.to_numpy()
    train_y_arr = train_y.to_numpy().flatten()
    test_X_arr = test_X.to_numpy()
    test_y_arr = test_y.to_numpy().flatten()
    
    # Convert numpy arrays to PyTorch tensors
    train_X_tensor = torch.tensor(train_X_arr, dtype=torch.float32)
    train_y_tensor = torch.tensor(train_y_arr, dtype=torch.long) 
    test_X_tensor = torch.tensor(test_X_arr, dtype=torch.float32)
    test_y_tensor = torch.tensor(test_y_arr, dtype=torch.long)
    
    print("your dataframe has been splitted to a traing and testing dataset as well as it has been converted to  for furthe preprocessing")
    
    return train_X_tensor,train_y_tensor,test_X_tensor,test_y_tensor

In [207]:
## Doing a train test split in 80:20 ratio

train_X_tensor,train_y_tensor,test_X_tensor,test_y_tensor = data_preprocessing("iris.csv",80,20,"variety")

The shape of your dataset is : (150, 5)
The shape of your traing data set is: (120, 4)
The shape of your testing data set is : (30, 4)
your dataframe has been splitted to a traing and testing dataset as well as it has been converted to  for furthe preprocessing


In [208]:
# Create TensorDataset
train_dataset = TensorDataset(train_X_tensor, train_y_tensor)
test_dataset = TensorDataset(test_X_tensor, test_y_tensor)
    
# Create DataLoader
train_loader = DataLoader(train_dataset)
test_loader = DataLoader(test_dataset)

In [209]:
class Artificial_NN_A(nn.Module):
    
    def __init__(self):
        super(Artificial_NN_A, self).__init__()
        self.hidden_layers = nn.Sequential(
            nn.Linear(4,20),
            nn.Sigmoid(),
            nn.Linear(20,20),
            nn.Sigmoid(),
            nn.Linear(20,3)  
        )
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self, input_d):
        hidden_outcome = self.hidden_layers(input_d)
        final_predictions = self.softmax(hidden_outcome)
        return final_predictions

In [210]:
def Train_Network(num_of_epochs, learning_rate, training_set):
    
    ANN = Artificial_NN_A()  

    # choosing the cross entropy loss
    loss_fn = nn.CrossEntropyLoss()

    # choose ADAM optimizer and learning rate
    optimizer = optim.Adam(ANN.parameters(), lr=learning_rate)
    
    for epoch in range(num_of_epochs+1):
        
        for inputs, labels in training_set:
            
            optimizer.zero_grad()
            
            predictions = ANN(inputs)   # make predictions
            
            calc_loss = loss_fn(predictions, labels)  # calculate loss
            
            # Backprop
            calc_loss.backward()
            
            # update all the parameters
            optimizer.step()
                
    return ANN  # return the model

In [211]:
m = Train_Network(100, 0.001,train_loader)

In [212]:
def Test_Network(test_set, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_set:
            predictions = model(inputs)
            _, predicted = torch.max(predictions.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = (correct / total)*100
    misclafication_rate = 100 - accuracy
    print(f'misclassification rate on the given data set: {misclafication_rate}')
    return misclafication_rate

In [213]:
Test_Network(test_loader,m)

misclassification rate on the given data set: 3.3333333333333286


3.3333333333333286